# Basics

Let's first take a look at what's inside the ``ib_insync`` package:

In [1]:
import ib_insync
print(ib_insync.__all__)

['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescrip

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_insync import *
util.startLoop()

*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [3]:
ib = IB()
ib.connect('127.0.0.1', 4002, clientId=21)

<IB connected to 127.0.0.1:4002 clientId=21>

In [4]:
t = ib.openTrades()

In [5]:
t

[]

In [6]:
nq = ContFuture('nq', 'globex')
ib.qualifyContracts(nq)

[ContFuture(conId=396335999, symbol='NQ', lastTradeDateOrContractMonth='20210319', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH1', tradingClass='NQ')]

In [8]:
bracket = ib.bracketOrder('BUY', 2, 12315, 12385, 12280, outsideRth=True)

In [9]:
bracket

BracketOrder(parent=LimitOrder(orderId=4, action='BUY', totalQuantity=2, lmtPrice=12315, transmit=False, outsideRth=True), takeProfit=LimitOrder(orderId=5, action='SELL', totalQuantity=2, lmtPrice=12385, transmit=False, parentId=4, outsideRth=True), stopLoss=StopOrder(orderId=6, action='SELL', totalQuantity=2, auxPrice=12280, parentId=4, outsideRth=True))

In [10]:
for o in bracket:
    ib.placeOrder(nq, o)

In [12]:
trades =ib.openTrades()

In [13]:
orders = [t.order for t in trades]
orders

[LimitOrder(orderId=4, clientId=21, permId=902949114, action='BUY', totalQuantity=2, lmtPrice=12315, transmit=False, outsideRth=True),
 LimitOrder(orderId=5, clientId=21, permId=902949115, action='SELL', totalQuantity=2, lmtPrice=12385, transmit=False, parentId=4, outsideRth=True),
 StopOrder(orderId=6, clientId=21, permId=902949116, action='SELL', totalQuantity=2, auxPrice=12280, parentId=4, outsideRth=True)]

In [16]:
status = [t.orderStatus for t in trades]
status

[OrderStatus(orderId=4, status='Submitted', filled=0.0, remaining=2.0, avgFillPrice=0.0, permId=902949114, parentId=0, lastFillPrice=0.0, clientId=21, whyHeld='', mktCapPrice=0.0),
 OrderStatus(orderId=5, status='PreSubmitted', filled=0.0, remaining=2.0, avgFillPrice=0.0, permId=902949115, parentId=4, lastFillPrice=0.0, clientId=21, whyHeld='child', mktCapPrice=0.0),
 OrderStatus(orderId=6, status='PreSubmitted', filled=0.0, remaining=2.0, avgFillPrice=0.0, permId=902949116, parentId=4, lastFillPrice=0.0, clientId=21, whyHeld='child,trigger', mktCapPrice=0.0)]

Peer closed connection


If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [5]:
ib.positions()

[Position(account='DU1542749', contract=Forex('EURUSD', conId=12087792, localSymbol='EUR.USD', tradingClass='EUR.USD'), position=100.0, avgCost=1.1459)]

In [20]:
ib.tickers()

[Ticker(contract=ContFuture(conId=396335999, symbol='NQ', lastTradeDateOrContractMonth='20210319', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH1', tradingClass='NQ'), time=datetime.datetime(2020, 12, 22, 11, 26, 59, 873560, tzinfo=datetime.timezone.utc), bid=12734.0, bidSize=16, ask=12734.75, askSize=2, last=12734.25, lastSize=1, prevBid=12735.75, prevBidSize=2, prevAsk=12736.5, prevAskSize=3, prevLast=12736.0, prevLastSize=2, volume=67001, open=12698.25, high=12740.5, low=12639.5, close=12683.5, halted=0.0),
 Ticker(contract=ContFuture(conId=396336057, symbol='MNQ', lastTradeDateOrContractMonth='20210319', multiplier='2', exchange='GLOBEX', currency='USD', localSymbol='MNQH1', tradingClass='MNQ'), time=datetime.datetime(2020, 12, 22, 11, 28, 39, 591891, tzinfo=datetime.timezone.utc), bid=12735.5, bidSize=3, ask=12736.25, askSize=3, last=12736.0, lastSize=1, volume=120179, open=12697.5, high=12741.75, low=12639.75, close=12683.5, halted=0.0)]

In [23]:
ib.ticker(nq)

Ticker(contract=ContFuture(conId=396335999, symbol='NQ', lastTradeDateOrContractMonth='20210319', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQH1', tradingClass='NQ'), time=datetime.datetime(2020, 12, 22, 11, 45, 35, 102421, tzinfo=datetime.timezone.utc), bid=12728.25, bidSize=1, ask=12728.75, askSize=4, last=12728.25, lastSize=1, prevBid=12734.0, prevBidSize=16, prevAsk=12734.75, prevAskSize=2, prevLast=12734.25, prevLastSize=2, volume=68127, open=12698.25, high=12740.5, low=12639.5, close=12683.5, halted=0.0)

In [35]:
ib.reqTickers()

[]

In [11]:
mnq = ContFuture('mnq', 'globex')
ib.qualifyContracts(mnq)

[ContFuture(conId=396336057, symbol='MNQ', lastTradeDateOrContractMonth='20210319', multiplier='2', exchange='GLOBEX', currency='USD', localSymbol='MNQH1', tradingClass='MNQ')]

In [38]:
ib.reqTickers(mnq)

[Ticker(contract=ContFuture(conId=396336057, symbol='MNQ', lastTradeDateOrContractMonth='20210319', multiplier='2', exchange='GLOBEX', currency='USD', localSymbol='MNQH1', tradingClass='MNQ'), time=datetime.datetime(2020, 12, 22, 13, 37, 4, 107131, tzinfo=datetime.timezone.utc), bid=12746.75, bidSize=3, ask=12747.0, askSize=1, last=12747.0, lastSize=1, prevBid=12726.25, prevBidSize=4, prevAsk=12726.75, prevAskSize=2, prevLast=12726.25, prevLastSize=2, volume=140247, open=12697.5, high=12760.0, low=12639.75, close=12683.5, halted=0.0, ticks=[TickData(time=datetime.datetime(2020, 12, 22, 13, 37, 4, 107131, tzinfo=datetime.timezone.utc), tickType=1, price=12746.75, size=3), TickData(time=datetime.datetime(2020, 12, 22, 13, 37, 4, 107131, tzinfo=datetime.timezone.utc), tickType=2, price=12747.0, size=1), TickData(time=datetime.datetime(2020, 12, 22, 13, 37, 4, 107131, tzinfo=datetime.timezone.utc), tickType=4, price=12747.0, size=1), TickData(time=datetime.datetime(2020, 12, 22, 13, 37, 4,

In [33]:
ib.realtimeBars()

[]

In [26]:
dir(ib.accountValues()[0])

['__add__',
 '__annotations__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_field_types',
 '_fields',
 '_fields_defaults',
 '_make',
 '_replace',
 'account',
 'count',
 'currency',
 'index',
 'modelCode',
 'tag',
 'value']

Or filter the account values to get the liquidation value:

In [14]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU1542749', tag='NetLiquidationByCurrency', value='1003407.77', currency='BASE', modelCode='')]

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [7]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [8]:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='STK', conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,FOXRIVER,NYSENAT,PSX', priceMagnifier=1, longName='TESLA INC', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST5EDT', tradingHours='20190711:0400-20190711:2000;20190712:0400-20190712:2000;20190713:CLOSED;2019

### Current state vs request

Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So it is preferable to use the current state methods over requests. For example, use ``ib.openOrders()`` in preference over ``ib.reqOpenOrders()``, or ``ib.positions()`` over ``ib.reqPositions()``, etc:

In [9]:
%time l = ib.positions()

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 11.4 µs


In [10]:
%time l = ib.reqPositions()

CPU times: user 1.01 ms, sys: 0 ns, total: 1.01 ms
Wall time: 3.44 ms


### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [11]:
util.logToConsole()

To see all debug messages (including network traffic):

In [12]:
import logging
util.logToConsole(logging.DEBUG)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [14]:
ib.disconnect()